# Azure Model Deploy
In this notebook we will deploy our Neural Image Captioning Model to an Azure Web Service using Azure ML API for Python, at the end of this notebook we will have a single endpoint we can query with a POST Request to caption an image.

## Dependencies
Let's begin installing the modules *azureml* and *azureml-core* using pip.

In [0]:
!pip install azureml
!pip install azureml-core

## Create the Workspace
First, we have to create a new Workspace, put your Azure Subscription ID inside *subscription_id*.

In [0]:
from azureml.core import Workspace

ws = Workspace.create(name='ImageCaptioningWorkspace',
                      subscription_id='<YOUR_SUBSCRIPTION_ID'>,
                      resource_group='M',
                      create_resource_group=True,
                      location='eastus2', 
                      exist_ok=True)

Deploying KeyVault with name imagecapkeyvaultd350883b.
Deploying StorageAccount with name imagecapstorage930027103.
Deploying AppInsights with name imagecapinsights4fcdb66c.
Deployed AppInsights with name imagecapinsights4fcdb66c. Took 7.31 seconds.
Deployed KeyVault with name imagecapkeyvaultd350883b. Took 27.69 seconds.
Deployed StorageAccount with name imagecapstorage930027103. Took 31.65 seconds.
Deploying Workspace with name ImageCaptioningWorkspace.
Deployed Workspace with name ImageCaptioningWorkspace. Took 58.62 seconds.


## Register the Model
Our entire model is composed by three sub-models:
* The tokenizer: in pickle format
* The image encoding model: encoded in h5 format
* The image captioning model: encoded in h5 format

We saved the models into the folder *model*, now it's time to register our Model using this directory.

In [0]:
from azureml.core.model import Model

model = Model.register(model_path = "./model",
                       model_name = "ImageCaptioningModel",
                       description = "An Image Captioning model",
                       workspace = ws)

Registering model ImageCaptioningModel


## Define the Estimator
We have to define a script that will be executed when our API is called, this script contains just two functions:
* **init**: it is used to load models and initialize stuff that we need (if we need)
* **run** it receives the request' payload in json format, which is supposed to contain the image encoded in some way, and here we will use our models to process inputs and generate captions.

We will accept images both from URL and list of flattened pixels. 


In [0]:
%%writefile caption.py

import pickle
import json
import sys

from azureml.core.model import Model

import numpy as np

from keras.models import load_model
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.image import img_to_array

from PIL import Image
import requests
from io import BytesIO

import traceback


def init():
    
    global caption_model
    global tokenizer
    global encode_model
    global model_path
    
    global MAX_LEN
    global OUTPUT_DIM
    global WIDTH
    global HEIGHT
    
    MAX_LEN = 46
    OUTPUT_DIM = 2048
    WIDTH = 299
    HEIGHT = 299
        
    model_path = Model.get_model_path('ImageCaptioningModel')
    caption_model = load_model(model_path+"/caption_model.h5")
    encode_model = load_model(model_path+"/encode_model.h5")
    
    with open(model_path + '/tokenizer.pickle','rb') as handle:
        tokenizer = pickle.load(handle)



def run(raw_data):
    try:
        
        caption = "startseq"
        
        data = json.loads(raw_data)
         
        if("url" in data):
          
           # payload contains an URL
           # download the image
           # and convert to an array
          
            url = data["url"]
                
            response = requests.get(url)
            img = Image.open(BytesIO(response.content))
        
            img = img.resize((WIDTH, HEIGHT), Image.ANTIALIAS)  
            img = img_to_array(img)
            
        elif("data" in data):
          
            # payload contains a flattend array
            # reshape it with the correct dimensions
          
            arr = np.array(data["data"], dtype=np.float32)
            img = arr.reshape((WIDTH,HEIGHT, 3))
        else:
            # if no 'data' or 'url' is defined
            # return an error
            return {"error":"No data provided"}
        
        # preprocess the image
        
        img = preprocess_input(img)
        img = np.expand_dims(img, axis=0)
        
        x1 = encode_model.predict(img)
        x1 = x1.reshape((1, OUTPUT_DIM))
        
        # generate the caption
    
        for i in range(MAX_LEN):
            seq = tokenizer.texts_to_sequences([caption])
            x2 = pad_sequences(seq, maxlen=MAX_LEN)
        
            y = caption_model.predict([x1,x2], verbose=0)
            word = tokenizer.index_word[np.argmax(y)]
        
            if word == "endseq":
                break
      
            caption+=" "+word
    
        caption = caption.replace("startseq","").strip()
        return {"caption":caption}
    
    except Exception as e:
        print(traceback.format_exc())
        return {"error":str(e)}

Overwriting caption.py


## Define the Environment

To run our model we need to define the envirnment in which it will run, we can do this with a yml file. We can generate this file using the azureml's class *CondaDependencies*. Our model needs the following dependencies:

* **Numpy**: for arrays manipulation
* **Pillow**: for images processing
* **Keras**: to load and use the neural network
* **Tensorflow**: to run keras.


In [0]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("numpy")
myenv.add_pip_package("azureml-core")
myenv.add_pip_package("Pillow")
myenv.add_pip_package("keras")
myenv.add_pip_package("tensorflow")

with open("mlenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

## Create the Container
Azure deploy ML models in a Docker container, let's build the image for the container.

In [0]:
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script = "caption.py",
                                                  runtime = "python",
                                                  conda_file = "mlenv.yml",
                                                  description = "A Neural Image Captioning Model Image"
                                                 )

image = ContainerImage.create(name = "image-captioning-container",
                              models = [model],
                              image_config = image_config,
                              workspace = ws)

image.wait_for_creation(show_output = True)

## Deploy the Model

It's time to deploy ! We configure our VM with 1 single CPU core and 1 GB of virtual memory, it will be enough.

In [0]:
from azureml.core.webservice import AciWebservice, Webservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 1, 
                                               tags = {"data": "image-captioning", "type": "classification"}, 
                                               description = 'An Image Captioning Model')

service = Webservice.deploy_from_image(deployment_config = aciconfig,
                                            image = image,
                                            name = "image-captioning",
                                            workspace = ws)



service.wait_for_deployment(show_output = True) # wait until deploy is completed

Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace



WebserviceException: WebserviceException:
	Message: Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace
	InnerException None
	ErrorResponse {"error": {"message": "Error, there is already a service with name image-captioning found in workspace ImageCaptioningWorkspace"}}

If we need we can print our service's log using the method *.get_logs()*

In [0]:
service.get_logs()

'2019-09-01T16:24:14,710416826+00:00 - gunicorn/run \n2019-09-01T16:24:14,716812803+00:00 - iot-server/run \n2019-09-01T16:24:14,731088576+00:00 - nginx/run \n2019-09-01T16:24:14,747645675+00:00 - rsyslog/run \nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2019-09-01T16:24:15,154286578+00:00 - iot-server/finish 1 0\n2019-09-01T16:24:15,206283304+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 19.9.0\nListening at: http://127.0.0.1:31311 (10)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 45\nInitializing logger\nStarting up app insights client\nStarting up request id generator\nStarting up app insight hooks\nInvoking user\'s init function\n2019-09-01 16:24:23,552 | azureml.core.run | DEBUG | Could not load run context RunEnvironmentException:\n\tMessage: Could not load a submitted run, if outside of an execution context, use experiment.start_logging to initialize an azureml.core.Run.\n\tInnerExcepti

If we have already created the service and we need to update it, then we can use the method *.update(image)* with the new image to deploy.

In [0]:
service = Webservice(name="image-captioning", workspace = ws)
service.update(image=image)

service.wait_for_deployment(show_output = True)

Running...

## Test the Web Service
Let's test our web service making a POST request to our service endpoint, first using a link to an image taken from Google Images.

In [1]:
import json
import requests

IMG_URL = "https://cdn.pixabay.com/photo/2017/02/20/18/03/cat-2083492_960_720.jpg"

input_data = json.dumps({"url": IMG_URL})

headers = {'Content-Type':'application/json'}

resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print(json.loads(resp.text))


NameError: ignored

and then with an image flattened in a list of pixel.

In [0]:
!wget https://images.financialexpress.com/2018/12/train-18-tracks-660.jpg

from keras.preprocessing.image import img_to_array
from PIL import Image

import json
import requests


img = Image.open("train-18-tracks-660.jpg")
img = img.resize((299, 299), Image.ANTIALIAS)  
img = img_to_array(img)

input_data = json.dumps({"data": img.tolist()})
headers = {'Content-Type':'application/json'}
resp = requests.post(service.scoring_uri, input_data, headers=headers)

print("POST to url", service.scoring_uri)
print(json.loads(resp.text))



### Our model is alive !

# How to improve our App
I use the web service we have just deployed from a mobile App called Pic2Speech, you can find it on [Google Play](https://play.google.com/store/apps/details?id=gfg.app.pictospeech). The app is still really simple, it just take a picture and call the APi to caption it, since the problem's complexity the app doesn't always return accurate results but it identificate the context of a picture most of the time. In future I would like to add the following functionalities to improve the app:

1. In the current status the app doesn't store pictures taken from users, in future would be great to ask users for feedbacks on generated captions and use that information to improve the model using **continual learning**.
1. Allow users to manually caption pictures taken by themself or by other users, who accepted to share their pictures with others, to improve the model using **continual learning**.
2. Add a metric to evaluate generated captions' accuracy, in the case of low accuracy the app will use a InceptionV3 model, even on device using Tensorflow Lite, to simply list the objects detected in the pictures. Then we could ask users to manually caption such pictures as in point 1.